In [1]:
import os #여기부터 실행하세요. one drive에서 파일 다운받아두세요. 
import re
import openpyxl
import calendar
from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import time

cal = calendar.Calendar()

query = "여성"
query2 = "#2_" + query
year = 2019
xlist = list()

for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
        for day in monthdays:
            dd = str(year) + str(month).zfill(2) + str(day).zfill(2)
            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#2_html2text', query2, '#2_'+ str(year) + '_'+query, 
                                     '{}.xlsx'.format("#2_"+dd+"_"+query))
            xlist.append(xlsx_name)


In [2]:
def replace_sub(text):
                
        from nltk.tokenize import regexp_tokenize
        from string import punctuation

        tlist, ttlist= list(), list()
        text = regexp_tokenize(text, r'[^\s\.][^\.\n]+')
        for _ in text:
            if len(_) > 11:
                tlist.append(_)
                
        text = ".\n".join(tlist) + "."

        text = re.sub(r'\d+\.(\n)\d+', r'', text)
        text = re.sub(r'\b(\w|[.])+@(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'\bhttps?://\w+(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-하-ㅣ\.?!\n ]', ' ', text)
        text = re.sub(r'\b[a-z][A-Za-z0-9]+\b', ' ', text)
        text = re.sub(r'\s{2,}', ' ', text) 
        text = re.sub(r'[\(\[].*?[\)\]]', '', text)
        text = re.sub('【.*】','',text)
        text = re.sub(r'"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\/:;<=>\@\\[\\\\\\]\\^_`\\{\\|\\}\\~', '', text)
        # 여기부터 새로운 태그 추가
        #신문사 이름은 나중에 한꺼번에 뺄 것이니 추가하지 말 것
        text = re.sub(r"-\d+-", '', text) # 이거 전에 '-'제거하면 안됨.
        text = re.sub(r"\b\w{3}\s기자", '', text)
        text = re.sub(r'\s?[a-z]\s', ' ', text)
        text = re.sub(r'\s+\d+px?', '', text)
        text = re.sub(r'\s\d+\s\d+\s\d+\s', '', text)
        text = re.sub(r'\s?(\d+\s)', '', text)
        text = re.sub(r'((\d+[a-z])+)', '', text)
        text = re.sub(r'[A-Z0-9a-z][A-Z0-9a-z]', '', text)
        text = re.sub(r'\b[A-Z0-9]\b', '', text)
        
        text = re.sub('해당 언론사가.*?가시겠습니까?','',text,flags=re.DOTALL)
        text = re.sub('무단전재.*?분류했습니다.','',text,flags=re.DOTALL)
        text = re.sub('기사 섹션 분류 안내.*?따르고 있습니다.','',text,flags=re.DOTALL)
        text = re.sub('모두에게 보여주고.*?활용됩니다.','',text,flags=re.DOTALL)
        text = re.sub('네이버에서.*?클릭','',text,flags=re.DOTALL)
        text = re.sub('제한된 글자수 자를.*?비밀번호 가 일치 하지 않습니다.','',text,flags=re.DOTALL)
        
        text = text.replace('출처', '').replace('방송캡처', '').replace('-', '').replace('오류를 우회하기 위한 함수 추가.', '')
        text = text.replace('동영상 뉴스.', '').replace('아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.', '')
        text = text.replace(' 사진 ', ' ').replace('경남신문의 콘텐츠는 저작권법의 보호를 받는 바 무단전재 복사 재배포를 금합니다.', '')
        text = text.replace('돋움', '').replace('메인이미지', '').replace('. p .', "\n").replace('정상처리되지 않았습니다.', '')
        text = text.replace(' p ', '\n').replace('IMGFLOATING ', '').replace('캡쳐 ', '').replace(' P ', '').replace('INSERT RANDOM NUMBER HERE', '')
        text = text.replace('OptimaA Scrip Ta', '').replace('시원하게 털어드립니다.', '').replace('저작권자 인사이트 무단전재 및 재배포 금지 .', '')
        text = text.replace('수직배너 와이드', '').replace('www.', '').replace('동영상이 안 나오면', '').replace('여기를 누르시면 동영상을 별도의 새 창에서 볼 수 있습니다.', '')
        
        text = text.replace('취재기자 심충만 방송일조회', '').replace('게시글을 r로 보내기', '').replace('게시글을 으로 보내기', '').replace('게시글을 로 보내기', '')
        text = text.replace('이 글을 페이스북으로 퍼가기', '').replace('이 글을 트위터로 퍼가기', '').replace('이 글을 카카오스토리로 퍼가기', '').replace('이 글을 밴드로 퍼가기', '')
        text = text.replace('요즘 게시글을 요즘으로 보내기', '').replace('E에 있는 정보를 기본으로 가져오며', '').replace('해당 테이블에 등록된 정보가 없을 시', '')
        text = text.replace('모바일에 등록된 4를 가져온다', '').replace('롱리브더킹 임시 해결', '').replace('오른쪽 마우스 부정클릭 방지', '')
        text = text.replace('카툰 유료화 안내 표시', '').replace('기사본문 광고 제거', '').replace('! !','').replace('? ?', '')
        text = text.replace('기자의 다른 기사 보기','').replace('뉴광고 기사 배너지면 가변','').replace('기사 모바일 우측상단','')
        text = text.replace('게시물을 삭제하시겠습니까?','').replace('로그인하신 후에 이용해 주시기 바랍니다!','').replace('본문상단우측','').replace('공식 이미지','')
        text = text.replace('기자가 쓴 다른기사 보기','').replace('해당 언론사가 주요기사로 직접 선정한 기사입니다','').replace('언론사 편집판 바로가기','')
        text = text.replace('게티이미지뱅크','').replace('함수 를 구현합니다','').replace('언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다','')
        text = text.replace('헉! 소리나는 스!토리 뉴스 헉스','').replace('클릭해 뉴스 들어간다 뉴스쿨','').replace('이 기사를 추천합니다','')
        text = text.replace('본문 텍스트A','').replace('포토 위젯 시작 문의','').replace('포토 위젯 종료 문의','').replace('게티이미지','')
        text = text.replace('이전데이터에서 사진이 추가될 경우 본문 삽입처리','').replace('기사본문 오른쪽 정렬','')
        text = text.replace('본문내 상단우측','').replace('아티클 공통 기자','').replace('아티클 공통 공유 및 댓글','')
        text = text.replace('새창 페이스북으로 이동 페이스북','').replace('새창 트위터로 이동 트위터','').replace('새창 카카오스토리로 이동 카카오스토리','')
        text = text.replace('댓글로 이동 댓글 수','').replace('아티클 공통 공유 및 댓글','').replace('아티클 공통 본문','').replace('아티클 공통 부제','')
        text = text.replace('기사 페이지 본문내우측','').replace('이미지 확대보기','').replace('에스엔에스 공유경제신문','').replace('에스엔에스에스비에스화면','') 
        
        text = text.replace('저작권자 입법국정전문지 더리더 e 무단전재 및 재배포 금지 .','').replace('머니투데이 더리더','')
        text = text.replace('사진','').replace('에너지경제 온에어','').replace('뉴스','')
        text = text.replace('제공 안양시 안양 파이낸셜','').replace('매직스트로베리사운드 제공.','')
        text = text.replace('정하은기자','').replace('캡처 헤럴드P','').replace('경기도청 전경 군포시민신문.','').replace('독자가 내는 소중한 월 1천원 구독료는 군포시민신문 대부분의 재원이자 올바른 지역언론을 지킬 수 있는 힘입니다.','')
        text = text.replace('충북 제천경철서 홈페이지 캡처.','').replace('페이스북 캡처','').replace('김민성 인턴기자','').replace('S썸바이벌캡처','')
        text = text.replace('저작권자 스타트업투데이 무단전재 및 재배포 금지.','').replace('로그인 후 이용 가능합니다.','').replace('인스타그램 캡처','')
        text = text.replace('이 에디터의 다른 글 보기.','').replace('저작권자 아트인사이트 .','').replace('추천 비추천은 회원만 가능합니다.','').replace('값이 제대로 넘어오지 않았습니다.','').replace('등록된 코멘트가 없거나 코멘트 글이 아닙니다.','')
        text = text.replace('자신의 글에는 추천 또는 비추천 하실 수 없습니다.\n이미 추천 하신 글 입니다.\n이미 비추천 하신 글 입니다.','').replace(' 페이스북 페이지 캡처','')
        text = text.replace('디지털 콘텐츠 이용규칙 보기','').replace('조혜진기자 앵커','').replace('권은주기자','').replace('! 뉴데일리 뉴데일리','').replace('출품 여성작가들과 임종국 교수 오른쪽','')
        text = text.replace('출품 작가 김미숙 김종태 신보경 신소윤 안선문 인상희 임순옥 임윤상 한경진 홍은희 홍태돈 지도교수 임종국 .','')
        text = text.replace('전시회는 7월8일까지 서울교대 샘미술관 아카데미 실용반 활동영상 .','').replace('E평생교육원실용반그룹전 g영상','').replace('A아래에 작가별 대표작들을 소개한다','')
        text = text.replace('무단전재 재배포 금지','').replace('보도자료 및 기사제보','').replace('자유민주 시장경제의 파수꾼 뉴데일리 ','').replace('기사','')
        text = text.replace('인터플레이 중앙중단','')
        text = text.replace('라이브리 댓글 작성을 위해 를 활성화 해주세요','').replace('뉴데일리 댓글 운영정책','')
        text = text.replace('왼쪽부터 임호천씨박주식씨김해정씨.','').replace('이상훈 우철훈 김정근 선임기자','').replace('제공 e','')
        text = text.replace('우창범 트위터 웹데일리','').replace('오애희 웹데일리 기자','')
        text = text.replace('저작권자 웹데일리 무단 전재 및 재배포 금지 .','').replace('저작권자 기호일보 무단전재 및 재배포 금지','').replace('정상처리되지 않았습니다!','')
        text = text.replace('원하는곳에 붙여넣기','').replace('하루 김명민 이종석과 다정하게 한 컷','').replace(' 모바일 우측상단','').replace('김유진 공유경제신문 기자','').replace('공유경제가 제공하는 콘텐츠에 대해 독자는 친근하게 접근할 권리와 정정ㆍ반론ㆍ추후 보도를 청구 할 권리가 있습니다','').replace('저작권자 공유경제신문 무단 전재 및 재배포 금지','')
        text = text.replace('이건희 공유경제신문 기자','').replace('기부와 문화인류학 기부의 성립조건','').replace('공유 시선 공동체 자치 가 공유지의 비극 해결','')
        text = text.replace('공유 시선 서구 문학에서 장애는 어떻게 재현되나.\n공경기획 기업의 공유가치창출 기업가치를 높이는 경영전략.','').replace('산책 선순환 방식의 기부문화 나와야','')
        text = text.replace('기부와 문화인류학 기부와 경제.','').replace('공경기획 공유경제 플랫폼의 다양화로 협력적 소비 이끌어','').replace('산책 프로보노 사회의 변화를 추구하는 기부 방식','')
        text = text.replace('공경기획 공유경제 서비스 영역의 확대','').replace('공경기획 숙박 공유 플랫폼 지역사회 경제 기여','').replace('공경기획 공유경제는 소유에서 시작된다','')
        text = text.replace('저작권자 대구 경북 대표지역언론 대구일보 ','').replace('폴리 박현 기자','').replace('월 일모바일한경 에 게재된 입니다','').replace('모바일한경 더보기 ','').replace('모바일한경은 프리미엄 채널입니다','').replace('한국정책방송원 무단전재 및 재배포 금지','').replace('대한 9회 클립영상','').replace('영상취재 김명신 홍성주 영상편집 양세형','')
        text = text.replace('저작권자 로이슈 무단 전재 및 재배포 금지','').replace('제보 및 보도자료 제공','').replace('에디터에서 들어간 가 아닐 경우 태그 씌우지 않음','').replace('이 는 언론사에서 사회 섹션으로 분류했습니다','').replace('네이버에서 뉴시스 채널 구독하기 뉴시스 빅데이터 주가시세표 바로가기 저작권자 공감언론 뉴시스통신사','').replace('의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다','').replace('이 는 사용자 추천 으로 모바일 메인 판에 노출된 이력이 있습니다','')
        text = text.replace('무단전재 및 재배포 금지','').replace('저작권자 헤드라인제주','')
        text = text.replace('메인페이지가 로드됩니다','').replace('본문 중하단 광고 서인선','').replace('의견글을 삭제하시겠습니까?','').replace('제공 의정부시 의정부 스포츠서울','').replace('후원 금액을 선택해주세요','').replace('정기후원 페이지로 이동합니다','')
        text = text.replace('팝업 차단를 해제하고 이용하시기 바랍니다','').replace('팝업창을 활성화 해주세요','').replace('가 마음에 드셨나요? ','').replace('학교비정규직노조 울산지부 제공','').replace('문체부 게시물 분석와 무관한 이미지입니다','').replace('이미지 로이터연합','').replace('류원혜 인턴기자','').replace('김고금평 기자','').replace('타다 문제 계약해지 성 인지 교육 강화 저작권자','').replace('무단복제재배포 금지 네이버 홈에서 C 채널 구독하기 엠빅분토론 심상정양경규 진보에 길을 묻다 살충제 이겨낸 바퀴벌레의 초특급 진화','').replace('이 는 모바일 메인 판 에 노출된 이력이 있습니다','')
        text = text.replace('제한된 글자수 자를 초과하였습니다','').replace('닉네임 을 입력해 주세요','').replace('비밀번호 를 입력해 주세요','').replace('건전한 댓글문화 정착을 위해 이용에 주의를 부탁드립니다','').replace('명예훼손 개인정보 유출 인격권 침해 허위사실 유포 등은 이용약관 및 관련법률에 의해 제재를 받을 수 있습니다.','').replace('댓글내용 을 입력해 주세요','').replace('양수미 공유경제신문 기자','').replace('양수미 공유경제신문 기자','').replace('무단 전재 및 재배포 금지','').replace('꿈을 담는 캔버스 채널A ','').replace('저작권 언론사 하이닥 무단 전재 및 재배포 금지','').replace('저작권자 패션엔미디어','').replace('저작권자 중앙일보플러스 무단전재 및 재배포금지 ','').replace('채널A 무료 보기 네이버에서 채널A 구독하기 보기 판결의 재구성 모자 실종 반전의 잔혹극 신고자가 살해 용의자','')
        text = text.replace('김수지기자','').replace('무단 전재재배포 금지','').replace('공보닷컴 트위터 캡처 헤럴드경제 정지은 인턴기자','').replace('이미지 연합 아시아경제 조슬기나 기자','')
        text = text.replace('제공 삼성화재 스포츠서울 삼성화재는 어','').replace(' 열매 아프리카 방송 캡처온라인','').replace('저작권 한국일보','').replace('이지웅 웹데일리 기자','')
        text = text.replace('글 이혁재 소아시한의원 원장 주간경향','').replace('글 이혁재 소아시한의원 원장 주간경향','').replace('의 디지털이용규칙에 따른 저작권을 행사합니다','')
        text = text.replace('제공 아메바컬쳐아메바컬쳐','').replace('위 의 법적인 책임과 권한은 에듀동아에 있습니다','').replace('에듀동아 전수완 인턴 기자','').replace('인라이플 본문 삽입','').replace('저작권자 무단복제재배포 금지','').replace('등록된 댓글이 없습니다','')
        text = text.replace('로그인 후 댓글 등록이 가능합니다','').replace('지금 로그인 하시겠습니까','').replace('주소를 클립보드에 복사하시겠습니까?','').replace('이 글의 L주소입니다.','').replace('를 눌러 클립보드로 복사하세요','').replace('비즈 인터플레이 중앙중단','')
        text = text.replace('보도자료제목','').replace('제공 아메바컬쳐힙합그룹','').replace('스포츠서울 김자영기자','').replace('로이슈가 제공하는 콘텐츠에 대해 독자는 친근하게 접근할 권리와 정정ㆍ반론ㆍ추후 보도를 청구 할 권리가 있습니다.','')
        
        
        
        
        
        
        text = text.replace('C뉴스 제희원입니다.','').replace('취재기자 제희원 방송일조회.','').replace('뉴스돋보기 여성장애인으로 산다는 것 .','')
        text = text.replace('연합뉴스 김동욱입니다.','').replace('방송화면 캡처','').replace('방송 영상 캡처','').replace('윤소윤 인턴기자','').replace('최진실기자','')
        text = text.replace('지원하지 않는 브라우저 입니다.','').replace('백호튜터링','').replace('트위터 제공','').replace('?제공','')
        text = text.replace('트위터 보내기 트위터.','').replace('페이스북 보내기 페이스북.','').replace('네이트온 보내기 네이트온 보내기.','')
        text = text.replace('제작 이태호 연합','').replace('영상취재 김태형 영상편집 양세형 .','')
        text = text.replace('인사이트','').replace('배너지면','').replace('라이브리 프리미엄 설치코드','').replace('선거전 기존 코드','')
        text = text.replace('면 경제섹션','').replace('?p 뉴시스 웹데일리 .','').replace('신세계푸드 광주기아챔피언스필드 신세계푸드 제공','').replace('류빈 기자','')
        try: 
            text = text[:text.rindex('다.')+2]
            text = text.split(sep='.')
            for _ in text:
                if len(re.sub(r'\s{2,}', ' ', _)) > 11:
                    ttlist.append(re.sub(r'\s{2,}', ' ', _))
                    
            text = ".".join(ttlist) + "."
            return text
        except:
            return 0


In [7]:
df = pd.read_excel(xlist[184])
print(len(df))

1505


In [12]:
"""
유미 0-91 지은 92-182 은서 183-273 유리 274-364
"""

df = pd.read_excel(xlist[184]) # 여기에 숫자 바꿔가면서 하면 돼
for i in range(100,150): # 한 번에 다 출력해서 하던가 아니면 하나하나 출력해서 하던가 해
    text = df.iloc[i, 3]
    print(replace_sub(text))


4일 서울 서초구 잠원동에서 철거 작업중이던 5층 건물 외벽이 무너지면서 깔린 부상자를 소방대원들이 구조하고 있다.
서울 서초구 신사역 인근 한 건물 일부가 무너져 도로를 덮치는 사고가 발생해 여성 1명이 숨졌다.
소방당국에 따르면 4일 오후 2시분쯤 서초구 잠원동 신사역 인근에서 철거 중이던 지상 5층 지하 1층짜리 상가 건물 가림막과 철골조 일부가 무너져 약잔해물이 도로를 덮쳤다.
이 사고로 시민 4명이 부상을 입고 지나가던 차량 2대와 주차된 차량 1대 등 차량 3대가 매몰돼 파손됐다.
한 차량에 있던 대 여성 2명은 경상을 입고 병원으로 옮겨져 치료를 받고 있다.
다른 차량에 있는 2명 중 운전자 남성 1명은 소방대원이 굴삭기 4대를 동원해 오후 5시분쯤 구조했다.
조수석에 있는 여성 1명은 의식을 잃어 수액을 맞은 후 구조됐지만 오후 6시분쯤 숨졌다.
주차돼 있던 나머지 차량에는 사람이 없었다.
소방 관계자는 여성 1명을 6시분쯤 구조 완료했지만 분쯤 서초보건소 의사가 사망 판정했다 고 전했다.

서울신문 부모의 동의가 성별 정정에 있어 필수가 아니라는 법원의 판단이 나왔다.
4일 공익 인권변호사 모임 희망을 만드는 법 희망법 에 따르면 인천가정법원은 부모 동의서를 제출하지 못한 대 후반 트랜스젠더 여성 A씨의 성별 정정 신청을 허가해 신청을 기각한 1심 결정을 뒤집었다.
A씨는 어린 시절부터 확고한 여성으로서의 성별 정체성을 갖고 현재 여성으로서 사회 생활을 하고 있다.
남성에서 여성으로의 성 전환 수술을 받는 등 법원의 성별 정정 허가를 받기 위한 요건을 모두 충족했다.
하지만 대법원 예규가 요구하는 서류 중 부모 동의서를 제출하지 못했다.
A씨 부모는 종교적인 이유로 A씨의 정체성을 거부했고 A씨는 부모와의 관계가 단절됐다.
1심인 인천가정법원 부천지원은 A씨의 성별 정정 신청을 기각했다.
결정문에는 기각 사유가 적혀 있지 않지만 A씨가 부모의 동의를 받지 못한 점 때문으로 추정된다고 희망법은 말했다.
하지만 2심 재판부는 원심 결정을 취소하고